<a href="https://colab.research.google.com/github/ytyimin/scm518/blob/main/Computer_Mix_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Product Mix - PC Tech Computers

## Objective and Prerequisites

PC Tech wants to maximize their profit by determining the number of  basic and XP type computers they should produce. The objectives of the product-mix problem are:

* The number of Computers produced should be less than demand,
* Assembly hours and Testing hours cannot exceed the available, and
* Ensure that the number of computers produced are valid, i.e., non- negative production.

---
## Problem Description

![picture](https://drive.google.com/uc?export=view&id=1K3Scp7yTfpKhYIYvz4epz4gpcZRhXKwa)


The company wants to decide how many of each model to assembly and then test.  Each Basic sells for 300 dollars and each XP sells for 450 dollars. The cost of
component parts for a Basic is 150; for an XP it is 225. Labor is required for assembly and testing. There are at most 10,000 assembly hours and 3000 testing hours available. Each labor hour for assembling costs 11 and each labor hour for testing costs 15. Each Basic requires five hours for assembling and one hour for testing, and each XP requires six hours for assembling and two hours for testing. PC Tech wants to know how many of each model it should produce (assemble and test) to maximize its net profit, but it cannot use more labor hours than are available, and it does not want to produce more than it can sell.

## Model Formulation

### Indices

$i \in \{B,X\}$: Index to represent different computer types.

### Parameters

$P_{i}$: Selling price of Computer type $i$.

$C_{i}$: Component cost of Computer type $i$.

$AC$: Assembly Cost of Computer.

$TC$: Testing Cost of Computer.

$LA_{i}$: Assembly hours Computer type $i$.

$LT_{i}$: Testing hours Computer type $i$.

$AA$: Available assembly hours.

$AT$: Available testing hours.

$D_{i}$: Demand of Computer type $i$.

### Decision Variables

$x_{i}$: Number of Computer type $i$ to produce .


### Objective Function

- **Profit**. We want to Maximize the total profit.


\begin{equation}
\text{Max}_{x_{i}} \quad \sum_{(i) \in {B,X}} [P_{i} - C_{i} -LA_{i}*AC -LT_{i}*TC ]*x_{i}
\tag{0}
\end{equation}

### Constraints

\begin{equation}
\ x_{i} \leq D_{i} \quad \ i \in \{B,X\} \quad (\text{number of computer type i cannot exceed demand})
\tag{1}
\end{equation}

\begin{equation}
\sum_{i \in (B,X)} LA_{i}*x_{i} \leq AA (\text{Assembly hours cannot exceed available})
\tag{2}
\end{equation}

\begin{equation}
\sum_{i \in (B,X)} LT_{i}*x_{i} \leq AT (\text{Testing hours cannot exceed available})
\tag{3}
\end{equation}

\begin{equation}
\ x_{i} \geq 0 \quad \ i \in \{B,X\} \quad (\text{Non - Negative production})
\tag{4}
\end{equation}


## Python Implementation

We now import the Gurobi Python Module

In [ ]:
pip install gurobipy

In [ ]:
import gurobipy as gp
from gurobipy import GRB

Set up the model

In [ ]:
#####################################################
#                    Model Formulation
#####################################################

m = gp.Model('computer mix')

In [ ]:
#Input Parameters
computer_type = [*range(0,2)]
Computer_type_label =['Basic','XP']

LaborAssembly = [5,6]
LaborTesting = [1,2]
CostComponents =[150,225]
SellingPrice = [300,450]
Demand = [600,1200]
TotalAssemblyhours = 10000
TotalTestinghours = 3000
LACost = 11
LTCost = 15


In [ ]:
#Decision Variable
x = m.addVars(computer_type, vtype=GRB.CONTINUOUS, name=Computer_type_label)

In [ ]:
# Set the Maximize Obijective: Total profit
m.setObjective(gp.quicksum((SellingPrice[i]-CostComponents[i]-LaborAssembly[i]*LACost -LaborTesting[i]*LTCost)*x[i] 
                          for i in computer_type),  GRB.MAXIMIZE)

In [ ]:
#Set Non-negative Decision Variable
c1 = m.addConstrs((x[i] >= 0 for i in computer_type), name = 'non-negative production')

# Total Computers produced must be less than or equal to Demand
c2 = m.addConstrs((x[i] <= Demand[i] for i in computer_type), name = 'cannot exceed maximum demand')

# Assemble hours cannot exceed available
c3 = m.addConstr((gp.quicksum(LaborAssembly[i]*x[i] for i in computer_type) <= TotalAssemblyhours), name = 'cannot exceed available assembly hours')

# Testing hours cannot exceed available
c4 = m.addConstr((gp.quicksum(LaborTesting[i]*x[i] for i in computer_type) <= TotalTestinghours), name ='cannot exceed available testing hours')

Solve the model

In [ ]:
# Run the optimize solver
m.optimize()

Results of the Model

In [ ]:
# Get the Optimal Solution for X
print("The Optimal number of each computer type to be produced for maximum profit\n")
m.printAttr('X')
def comma_value(number):
    return ("{:,}".format(number))
# print("\nTotal profit: $%8.2f" % (m.ObjVal))
print("\nTotal profit: $",comma_value(m.ObjVal))
# m.printAttr('RHS')
# m.printAttr('Slack')


More details on profits and resources used for each computer type

In [ ]:
#Finding amount of Profit generated by computer type
e = m.getObjective()
margin = [e.getCoeff(i) for i in computer_type ]
production = [v.x for v in m.getVars()]
revenue = [margin[i]*production[i] for i in computer_type]
for i in computer_type:
  print("Profit generated by",Computer_type_label[i],"computer is $",comma_value(revenue[i]))

In [ ]:
#Finding the number of assembly hours used by computer type
assembly_hour_used = [LaborAssembly[i]*production[i] for i in computer_type]
for i in computer_type:
  print("Number of Assembly hours used by",Computer_type_label[i],"computer is",comma_value(assembly_hour_used[i]))

print("Total Assembly hours used is",comma_value(sum(assembly_hour_used)))

In [ ]:
#Finding the number of testing hours used by computer type
testing_hour_used = [LaborTesting[i]*production[i] for i in computer_type]
for i in computer_type:
  print("Number of Testing hours used by",Computer_type_label[i],"computer is",comma_value(testing_hour_used[i]))

print("Total Testing hours",comma_value(sum(testing_hour_used)))

##  Conclusion

In the above problem we determined the number of computers that need to be produced to obtain the maximum profit keeping in mind the demand, assembly and testing hour constarints.
It is seen that the number of computers to be produced are 560 for Basic type and 1200 for the XP type


##  References
[1] Sixty examples of business optimization models. https://ytyimin.github.io/tart-cherry/.

[2] Gurobi python reference. https://www.gurobi.com/documentation/

[3] This example is developed by Shri Lekha K P www.linkedin.com/in/shri-lekha-k-p